In [28]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra
using Pkg
using IJulia
using Distributions

In [29]:
include("../Test Models/COVIDResourceAllocation.jl")
using .COVIDResourceAllocation

In [30]:
pct_nurses_available = 0.5
patients_per_nurse_covid = 2.5
nurse_hrs_per_week_covid = 36

@show nurse_hrs_per_day_covid = nurse_hrs_per_week_covid / 7
@show nurses_days_per_day_covid = 24 / nurse_hrs_per_day_covid

@show nurse_days_per_patient_day_covid = nurses_days_per_day_covid / patients_per_nurse_covid;

nurse_hrs_per_day_covid = nurse_hrs_per_week_covid / 7 = 5.142857142857143
nurses_days_per_day_covid = 24 / nurse_hrs_per_day_covid = 4.666666666666666
nurse_days_per_patient_day_covid = nurses_days_per_day_covid / patients_per_nurse_covid = 1.8666666666666665


In [31]:
states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
start_date = Date(2020, 4, 1)
end_date   = Date(2020, 5, 1)
travel_threshold_hours = 4.0
pct_beds_available = 0.25
travel_threshold_hours = 4.0
hospitalized_days = 14;

In [32]:
N = length(states);
T = (end_date - start_date).value + 1

forecast_active = forecast(
    states, start_date, end_date,
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
)
forecast_admitted = forecast(
    states, start_date,  end_date,
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
)
forecast_discharged = forecast(
    states, start_date-Dates.Day(hospitalized_days), start_date-Dates.Day(1),
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
)
forecast_discharged = hcat(forecast_discharged, zeros(Float32, N, T - hospitalized_days));

In [33]:
beds = n_beds(states, bed_type=:all, pct_beds_available=pct_beds_available);
adj = adjacencies(states, level=:state, source=:google, threshold=travel_threshold_hours);

In [74]:
demand = forecast_active * Float32(nurse_days_per_patient_day_covid);

In [35]:
nurses = n_nurses(states) * Float32(pct_nurses_available);

In [37]:
t  = 1
N = 11
contaminated_hospitals = [0,0,0,0,0,0,1,1,0,0,0]
indices = findall(x -> x == 1, contaminated_hospitals)

stage1_matrix = falses(N,N)

    for i in 1:N  
        for j in 1:N 
            # check that the edge is directed toward the contaminated hospital
            # if true set the matrix idx to 1 else leave it as 0 
            stage1_matrix[i,j] = (contaminated_hospitals[j] == 1)
        end
    end
    
stage2_matrix = falses(N,N)
for i in 1:N  
    for j in 1:N 
        # Manditorily make a fully connected graph among non-contaminated hospitals
        stage2_matrix[i,j] = ((contaminated_hospitals[i] == 0) && contaminated_hospitals[j] == 0)
    end
end

model1 = reusable_resource_allocation(
        nurses,
        zeros(Float32, size(demand[:, t:t+1])...),
        demand[:, t:t+1],
        stage1_matrix,
        obj_dir = :shortage,  
        send_new_only = false,
        sendrecieve_switch_time = 0,
        min_send_amt = 0,
        smoothness_penalty = 0,
        setup_cost = 0,
        sent_penalty = 0,
        verbose = true
    );
    
    sent_nurses_contam = value.(model1[:sent])
    indices = findall(x -> x == 1, contaminated_hospitals)
    received_by_contaminated = [sum(sent_nurses_contam[:, j, :]) for j in 1:length(contaminated_hospitals)]
    sent_to_contaminated = [sum(sent_nurses_contam[j, :, :]) for j in 1:length(contaminated_hospitals)]
    nurses_afterstage1 = nurses-sent_to_contaminated+received_by_contaminated

    forecast_initial = forecast(
    states, start_date-Dates.Day(t), start_date-Dates.Day(t),
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
    )[:]

model2 = patient_nurse_allocation(
        beds,
        forecast_initial,#demand[:, t:t+1],#nurses_afterstage1, 
        forecast_discharged[:, t:t+1],
        forecast_admitted[:, t:t+1],
        nurses_afterstage1,
        stage2_matrix,
        los=Exponential(1000),
        verbose =true,
    )

# sent_nurses_uncontam = value.(model2[:sentnurses])
# sent_patients_uncontam = value.(model2[:sentpatient])

println("termination status: ", termination_status(model2))
println("solve time: ", round(solve_time(model2), digits=3), "s")
println("objective function value: ", round(objective_value(model2), digits=3))
        

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-21
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E214)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 385 rows, 264 columns and 1386 nonzeros
Model fingerprint: 0xea63780d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+04]
Presolve removed 356 rows and 224 columns
Presolve time: 0.00s
Presolved: 29 rows, 40 columns, 96 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.3201240e+03   1.341083e+03   0.000000e+00      0s
      19    8.3201240e+03   0.000000e+00   0.000000e+00      0s

Solved in 19 iterations and 0.00 seconds (0.00 work units)
Optimal objective  8.320124023e+03

User-callback calls 78, time in user-callback 0.00 sec
Set parameter Username


In [131]:
# contaminated_hospitals = [0,0,0,0,0,0,1,1,0,0,0]
# stage1_matrix = falses(N,N)

#     for i in 1:N  
#         for j in 1:N 
#             # check that the edge is directed toward the contaminated hospital
#             # if true set the matrix idx to 1 else leave it as 0 
#             stage1_matrix[i,j] = (contaminated_hospitals[j] == 1)
#         end
#     end
    
# stage2_matrix = falses(N,N)
# for i in 1:N
#     for j in 1:N
#         # Mandatorily make a fully connected graph among non-contaminated hospitals
#         # Only set to true if both hospitals i and j are not contaminated
#         stage2_matrix[i, j] = (contaminated_hospitals[i] == 0) && (contaminated_hospitals[j] == 0)
#     end
# end



In [27]:
function contagion_model( 
    adj_matrix::BitArray{2}, 
    contaminated_hospitals::Array{<:Real,1}, 
    initial_nurses::Array{<:Real,1}, 
    demand::,
    initial_patients,
    discharged_patients,
    admitted_patients
    T::Real=0,
)
    #= 
        adj_matrix [N by N matrix]: specifies the edges between hospitals in a network; N is the number 
                    of hospitals in the network

        contaminated hospitals [array of size N]: an array where index i is 1 if the hospital is contaminated 
                                0 otherwise

        initial_capacities [array of size N]: an array where index i is the number of initial capacities in hospital i 
                            
        patients [N by 2 matrix]: initial patients at each hospital on day 1 and day 2

        T [int]: the time horizon
    =#

    # used to create new graph of the exact same size 
    N, T = size(demand)

    # construct the Stage 1 graph 

    stage1_matrix = falses(N,N)
    
    for i in 1:N  
        for j in 1:N 
            # check that the edge is directed toward the contaminated hospital
            # if true set the matrix idx to 1 else leave it as 0 
            stage1_matrix[i,j] = (contaminated_hospitals[j] == 1)
        end
    end

    # construct the Stage 2 graph 
    stage2_matrix = falses(N,N)
    for i in 1:N  
        for j in 1:N 
            # Manditorily make a fully connected graph among non-contaminated hospitals
            stage2_matrix = ((contaminated_hospitals[i] == 0) && contaminated_hospitals[j] == 0)
        end
    end

    for t in 1:T
        ###########
        # STAGE 1 #
        ###########
        # In stage 1 we only run resource redistribution model on a modified network.
        # The modified network is as follows: the only edges that exist are those that are directed to the contaminated
        # hosptials. In other words, we can only send resources to the contaminated hospitals in this stage. 
        # To accomplish this, we run a resource redistribtuion model. 
        
        # t is the step size of days we want to forward after each round

        model1 = reusable_resource_allocation(
            initial_nurses,
            zeros(Float32, size(demand[:, t:t+1])...),
            demand[:, t:t+1],
            stage1_matrix,
            obj_dir = :shortage,  
            send_new_only = false,
            sendrecieve_switch_time = 0,
            min_send_amt = 0,
            smoothness_penalty = 0,
            setup_cost = 0,
            sent_penalty = 0,
            verbose = true
        )
        
        sent_nurses_contam = value.(model1[:sent])
        sent_nurses_contam_matrix = sum(sent_nurses_contam, dims = 3)
        sent_nurses_contam_net_array = zeros(N)  # Ensure this dimension matches the number of hospitals
        for i in 1:N
            sent_nurses_contam_net_array[i] = sum(sent_nurses_contam_matrix[i, :]) - sum(sent_nurses_contam_matrix[:, i])
        end

        nurses_afterstage1 = initial_nurses + sent_nurses_contam_net_array
        
        ###########
        # STAGE 2 #
        ###########
        # In stage 2 we run a mixing patient and resource model on a modified network.
        # The modified network is as follows: no nodes are directed to the contaminated hosptials.
        # Uncontaminated hospitals are interconnected to each other. In other words, the goal is to balance the loads
        # between uncontaminated hospitals in this stage after helping the contaminated hospitals.
        # Note that the initial_nurses are initial_nurses before stage 1 - the number of nurses
        # sent to the contaminated_hospitals.
        # To accomplish this, we run a patient_nurse_allocation model.
        
        model2 = patient_nurse_allocation(
            beds = beds,
            initial_patients = initial_patients[:, t:t+1],
            discharged_patients = discharged_patients[:, t:t+1],
            admitted_patients = admitted_patients[:, t:t+1],
            initial_nurses = nurses_afterstage1,
            adj_matrix = stage2_matrix;
            los=11,
            nurse_days_per_patient_day=2.0,
            smoothness_penalty=0,
            setup_cost=0,
            sent_penalty=0,
            balancing_thresh_patients=1.0,
            balancing_penalty_patients=0,
            nurse_target_load=1.25,
            nurse_target_load_gap=0.25,
            nurse_load_penalty=0,
            disallow_nurse_shortage_sent=false,
            disallow_nurse_shortage_newpatients=false,
            severity_weighting=false,
            no_artificial_overflow=false,
            no_artificial_shortage=false,
            verbose =false,
        )

        sent_nurses_uncontam = value.(model2[:sentnurses])
        sent_patients_uncontam = value.(model2[:sentpatient])
        
        # initial value for the next round
        nurses = nurses_afterstage1 - sent_nurses_uncontam
        demand = demand - sent_patients_uncontam
        
        
        
        t = t+1

    end 
end

Base.Meta.ParseError: ParseError:
# Error @ /Users/joshuagrajales/Desktop/Healthcare-Resources-Optimization-main/scratch/contagion_model.ipynb:5:13
    initial_nurses::Array{<:Real,1}, 
    demand::,
#           ╙ ── unexpected `,`

In [ ]:
model2 = patient_nurse_allocation(
            beds,
            forecast_initial,#demand[:, t:t+1],#nurses_afterstage1, 
            forecast_discharged[:, t:t+1],
            forecast_admitted[:, t:t+1],
            nurses_afterstage1,
            stage2_matrix,
            los=Exponential(1000),
            verbose =true,
        )


In [80]:
for t in 1:T-1
    N = 11
    contaminated_hospitals = [0,0,0,0,0,0,1,1,0,0,0]
    indices = findall(x -> x == 1, contaminated_hospitals)

    stage1_matrix = falses(N,N)

        for i in 1:N  
            for j in 1:N 
                # check that the edge is directed toward the contaminated hospital
                # if true set the matrix idx to 1 else leave it as 0 
                stage1_matrix[i,j] = (contaminated_hospitals[j] == 1)
            end
        end
        
    stage2_matrix = falses(N,N)
    for i in 1:N  
        for j in 1:N 
            # Manditorily make a fully connected graph among non-contaminated hospitals
            stage2_matrix[i,j] = ((contaminated_hospitals[i] == 0) && contaminated_hospitals[j] == 0)
        end
    end

    model1 = reusable_resource_allocation(
            nurses,
            zeros(Float32, size(demand[:, t:t+1])...),
            demand[:, t:t+1],
            stage1_matrix,
            obj_dir = :shortage,  
            send_new_only = false,
            sendrecieve_switch_time = 0,
            min_send_amt = 0,
            smoothness_penalty = 0,
            setup_cost = 0,
            sent_penalty = 0,
            verbose = true
        );
        
        sent_nurses_contam = value.(model1[:sent])
        indices = findall(x -> x == 1, contaminated_hospitals)
        received_by_contaminated = [sum(sent_nurses_contam[:, j, :]) for j in 1:length(contaminated_hospitals)]
        sent_to_contaminated = [sum(sent_nurses_contam[j, :, :]) for j in 1:length(contaminated_hospitals)]
        nurses_afterstage1 = nurses-sent_to_contaminated+received_by_contaminated

        start_date = start_date + Dates.Day(1)
        forecast_initial = forecast(
        states, start_date-Dates.Day(1), start_date-Dates.Day(1),
        level=:state,
        source=:ihme,
        forecast_type=:active,
        patient_type=:regular,
        bound_type=:mean,
        )[:]

    model2 = patient_nurse_allocation(
            beds,
            forecast_initial,#demand[:, t:t+1],#nurses_afterstage1, 
            forecast_discharged[:, t:t+1],
            forecast_admitted[:, t:t+1],
            nurses_afterstage1,
            stage2_matrix,
            los=Exponential(1000),
            verbose =true,
        );

        sent_nurses_uncontam = value.(model2[:sentnurses])
        sent_patients_uncontam = value.(model2[:sentpatients])
        indices = findall(x -> x == 0, contaminated_hospitals)
        receivedn_by_uncontaminated = [sum(sent_nurses_uncontam[:, j, :]) for j in 1:length(contaminated_hospitals)]
        sentn_to_uncontaminated = [sum(sent_nurses_uncontam[j, :, :]) for j in 1:length(contaminated_hospitals)]
        receivedp_by_uncontaminated = [sum(sent_patients_uncontam[:, j, :]) for j in 1:length(contaminated_hospitals)]
        sentp_to_uncontaminated = [sum(sent_patients_uncontam[j, :, :]) for j in 1:length(contaminated_hospitals)]

        nurses = nurses_afterstage1-sentn_to_uncontaminated+receivedn_by_uncontaminated
        demand[:,t+1] .+= receivedp_by_uncontaminated - sentp_to_uncontaminated 


    # sent_nurses_uncontam = value.(model2[:sentnurses])
    # sent_patients_uncontam = value.(model2[:sentpatient])

    #println("termination status: ", termination_status(model2))
    #println("solve time: ", round(solve_time(model2), digits=3), "s")
    #println("objective function value: ", round(objective_value(model2), digits=3))
    print(t)
    t+=1
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-21
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E214)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 385 rows, 264 columns and 1386 nonzeros
Model fingerprint: 0x42f1adeb
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-02, 2e+04]
Presolve removed 385 rows and 264 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0783844e-02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  8.078384399e-02

User-callback calls 39, time in user-callback 0.00 sec
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-21
Gur